**프루닝 및 baseline 모델과의 속도차이 비교**

In [ ]:
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, time
import torch.nn as nn

# 필수 패키지 설치
!pip install -q "torch>=2.4.0" "transformers>=4.51.3" accelerate sentencepiece

# 허깅페이스 로그인
!huggingface-cli login

# 모델 로드
model_id = "google/gemma-3-1b-pt"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype=torch.float32,
    low_cpu_mem_usage=True,
    device_map="auto",
    attn_implementation="eager"
)

#레이어명 확인
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        print(name)

model.eval()
print(model.__class__.__name__, model.config.model_type, model.config.hidden_size)


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

**추론 함수 및 프루닝 전 모델 추론 속도 테스트**

In [ ]:
def quick_infer(model, tokenizer, prompt="Hello", max_new_tokens=30):
    model_cpu = model.to("cpu") #cpu로 런타임 유형 바꿔서 추론

    inputs = tokenizer(prompt, return_tensors="pt")
    start = time.time()
    with torch.no_grad():
        output = model_cpu.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False, temperature=0.1, top_p=0.8)
    elapsed = time.time() - start
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"\n--- Prompt ---\n{prompt}\n")
    print(f"--- Output ---\n{text}\n")
    print(f"[Elapsed] {elapsed:.2f} sec for {max_new_tokens} tokens on CPU\n")
    return elapsed

# 예시 실행 (프루닝 전 baseline 측정)
quick_infer(model, tokenizer, prompt="Q: What is the ideal daytime temperature for tomato plants?\nA", max_new_tokens=40)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- Prompt ---
Q: What is the ideal daytime temperature for tomato plants?
A

--- Output ---
Q: What is the ideal daytime temperature for tomato plants?
A: The ideal daytime temperature for tomato plants is between 70 and 80 degrees Fahrenheit. This range allows the plants to grow and develop properly, but it also prevents them from overheating or freezing

[Elapsed] 5.12 sec for 40 tokens on CPU



5.118504762649536

In [ ]:
def find_transformer_layers(model):
    container = getattr(model, "model", None) or getattr(model, "transformer", None) or model
    return getattr(container, "layers", None)

def get_mlp(block):
    mlp = block.mlp
    return mlp.gate_proj, mlp.up_proj, mlp.down_proj

@torch.no_grad()
def structured_prune_mlp(model, keep_ratio=0.7):
    layers = find_transformer_layers(model)
    gate0, up0, down0 = get_mlp(layers[0])
    inter = down0.in_features
    new_inter = max(16, int(round(inter * keep_ratio)))
    print(f"[INFO] intermediate {inter} → {new_inter} (keep {keep_ratio*100:.0f}%)")

    for li, block in enumerate(layers):
        gate, up, down = get_mlp(block)
        score = torch.norm(up.weight, dim=1) + torch.norm(gate.weight, dim=1)
        keep_idx = torch.topk(score, k=new_inter).indices.sort()[0]

        def clone_linear(old, new_out, new_in, sel_out=None, sel_in=None):
            new = nn.Linear(new_in, new_out, bias=old.bias is not None,
                            dtype=old.weight.dtype, device=old.weight.device)
            W = old.weight.data
            if sel_out is not None: W = W[sel_out, :]
            if sel_in is not None:  W = W[:, sel_in]
            new.weight.data.copy_(W)
            if old.bias is not None:
                b = old.bias.data
                if sel_out is not None: b = b[sel_out]
                new.bias.data.copy_(b)
            return new

        new_gate = clone_linear(gate, new_inter, gate.in_features, sel_out=keep_idx)
        new_up   = clone_linear(up,   new_inter, up.in_features,   sel_out=keep_idx)
        new_down = clone_linear(down, down.out_features, new_inter, sel_in=keep_idx)

        block.mlp.gate_proj, block.mlp.up_proj, block.mlp.down_proj = new_gate, new_up, new_down
        if li % 4 == 0:
            print(f"  - layer {li}: kept {new_inter}/{inter}")

    model.config.intermediate_size = new_inter
    print("[DONE] structured pruning complete.")

In [ ]:
model = model.to("cuda")
structured_prune_mlp(model, keep_ratio=0.7)

[INFO] intermediate 6912 → 4838 (keep 70%)
  - layer 0: kept 4838/6912
  - layer 4: kept 4838/6912
  - layer 8: kept 4838/6912
  - layer 12: kept 4838/6912
  - layer 16: kept 4838/6912
  - layer 20: kept 4838/6912
  - layer 24: kept 4838/6912
[DONE] structured pruning complete.


**프루닝된 모델 추론 속도 테스트**

In [ ]:
quick_infer(model, tokenizer, "Q: What is the ideal daytime temperature for tomato plants?\nA", max_new_tokens=40)


--- Prompt ---
Q: What is the ideal daytime temperature for tomato plants?
A

--- Output ---
Q: What is the ideal daytime temperature for tomato plants?
A in aviageur y in next a in 2, a 1 Kes a Fl- a 1ary a 1alARY The 1ate in and S of The 1

[Elapsed] 4.40 sec for 40 tokens on CPU



4.396235466003418

**프루닝된 모델을 LoRA 파인튜닝**

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

model = model.to("cuda")

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8, lora_alpha=16, lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "up_proj", "down_proj", "gate_proj"]
)
model = get_peft_model(model, lora_config)

In [ ]:
from datasets import load_dataset

# JSONL 파일 불러오기
dataset = load_dataset("json", data_files="/content/tomato_qa_3000.jsonl")["train"]
dataset = dataset.train_test_split(test_size=0.2, seed=42)   # train:eval = 80:20

train_dataset = dataset["train"]
eval_dataset  = dataset["test"]

# 토크나이징 함수 (batched=True 지원)
def tokenize_fn(examples):
    texts = [f"Q: {ins}\nA: {out} {tokenizer.eos_token}"
             for ins, out in zip(examples["instruction"], examples["output"])]
    return tokenizer(texts, truncation=True, max_length=128, padding="max_length")

train_dataset = train_dataset.map(tokenize_fn, batched=True)
eval_dataset  = eval_dataset.map(tokenize_fn, batched=True)

print(train_dataset[0])  # 토크나이즈 결과 확인

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

{'instruction': 'What fertilizer is best for tomato fruiting?', 'output': 'A balanced fertilizer with higher phosphorus supports fruiting.', 'input_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 236935, 236787, 2900, 46363, 563, 1791, 573, 30336, 42844, 4143, 236881, 107, 236776, 236787, 562, 20433, 46363, 607, 3715, 48262, 12765, 42844, 4143, 236761, 236743, 1], 'attention_mask': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
import wandb
import transformers
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling, EarlyStoppingCallback
from transformers.trainer_utils import IntervalStrategy

wandb.login()
wandb.init(project="tomato-qa", name="tomato-qa-pruning-lora")

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./lora-tomato",
    logging_dir="./logs",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    num_train_epochs=5,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="steps",
    save_steps=500,
    eval_strategy=IntervalStrategy.STEPS,
    eval_steps=500,
    report_to="wandb",
    run_name="tomato-qa-pruning-lora",
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print("🔗 대시보드 URL:", wandb.run.get_url())
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bewarmin (bewarmin-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


🔗 대시보드 URL: https://wandb.ai/bewarmin-/tomato-qa/runs/ms7ndtaw


Step,Training Loss,Validation Loss
500,0.083300,0.084921
1000,0.075700,0.076084
1500,0.074600,0.074356


TrainOutput(global_step=1500, training_loss=0.3494665530522664, metrics={'train_runtime': 973.8303, 'train_samples_per_second': 12.322, 'train_steps_per_second': 1.54, 'total_flos': 4762491715584000.0, 'train_loss': 0.3494665530522664, 'epoch': 5.0})

In [ ]:
quick_infer(model, tokenizer, "Q: What is the ideal daytime temperature for tomato plants?\nA", max_new_tokens=40)


--- Prompt ---
Q: What is the ideal daytime temperature for tomato plants?
A

--- Output ---
Q: What is the ideal daytime temperature for tomato plants?
A: The ideal daytime temperature is around 21-27°C (70-80°F). 

[Elapsed] 3.45 sec for 40 tokens on CPU



3.4463090896606445

**구글 드라이브에 사본 저장**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r ./lora-tomato /content/drive/MyDrive/

Mounted at /content/drive
